In [1]:
import os
os.chdir('../')

# 17. Future를 이용한 동시성

이 장에서는 `futures` 패키지를 통해 사용할 수 있는 `concurrent.futures` 라이브러리를 중점적으로 알아봅니다. 

## 17.1 예제: 세 가지 스타일의 웹 내려받기

긴 지연 시간 동안 cpu를 낭비하지 않기 위해 우리는 __동시성__을 이용해야 합니다. 

__네트워크에서 응답이 오는 동안 다른 일을 처리하는 것이 좋습니다.__

#### ex 17.1 flags.py, flags_threadpool.py, flags_asyncio.py

https://www.youtube.com/watch?v=A9e9Cy1UkME

위는 국기 이미지를 다운 받는 코드를 수행하는 예제입니다. 

국기 코드의 순서에 집중하면서 영상을 보면 되는데 동시성을 지원하는 `flags_threadpool.py` 나 `flags_asyncio.py` 를 실행하면 내려받을 때마다 국기의 순서가 달라지는 것을 확인할 수 있습니다. 

동시성을 활용하면 순차적으로 처리하는 것보다 빠른 속도를 보여줍니다.

>웹 서버에 동시에 수많은 HTTP 요청을 보내는 경우 의도치 않게 DoS 공격으로 인식될 수 있습니다. 따라서 상당히 부하를 주는 HTTP 클라이언트를 테스트하는 경우에는 직접 서버를 만드는 것이 좋습니다.

### 17.1.1 순차 내려받기 스크립트

In [5]:
! cat codes/flags.py

"""Download flags of top 20 countries by population

Sequential version

Sample run::

    $ python3 flags.py
    BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN
    20 flags downloaded in 10.16s

"""
# BEGIN FLAGS_PY
import os
import time
import sys

import requests  # <1>

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()  # <2>

BASE_URL = 'http://flupy.org/data/flags'  # <3>

DEST_DIR = 'downloads/'  # <4>


def save_flag(img, filename):  # <5>
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


def get_flag(cc):  # <6>
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content


def show(text):  # <7>
    print(text, end=' ')
    sys.stdout.flush()


def download_many(cc_list):  # <8>
    for cc in sorted(cc_list):  # <9>
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')

    re

- `requests` 라이브러리를 사용합니다.
- `download_many` <8> 코드가 동시성을 사용하는 코드와의 큰 차이점입니다. 단순히 for loop을 통해 순차적으로 국기를 다운받고 저장합니다.

### 17.1.2 `concurrent.futures`로 내려받기

`futures` 패키지의 가장 큰 특징

- `ThreadPoolExecutor`
- `ProcessPoolExecutor`

콜러블 객체를 여러 스레드나 프로세스에서 실행하는 인터페이스를 구현

In [6]:
!cat codes/flags_threadpool.py

"""Download flags of top 20 countries by population

ThreadPoolExecutor version

Sample run::

    $ python3 flags_threadpool.py
    BD retrieved.
    EG retrieved.
    CN retrieved.
    ...
    PH retrieved.
    US retrieved.
    IR retrieved.
    20 flags downloaded in 0.93s

"""
# BEGIN FLAGS_THREADPOOL
from concurrent import futures

from flags import save_flag, get_flag, show, main  # <1>

MAX_WORKERS = 20  # <2>


def download_one(cc):  # <3>
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))  # <4>
    with futures.ThreadPoolExecutor(workers) as executor:  # <5>
        res = executor.map(download_one, sorted(cc_list))  # <6>

    return len(list(res))  # <7>


if __name__ == '__main__':
    main(download_many)  # <8>
# END FLAGS_THREADPOOL


- <2> `ThreadPoolExecutor`에서 사용할 최대 스레드 수
- <5> 사용할 스레드 수를 executor에 전달해서 객체를 생성 `executor.__exit__()` 메서드는 `executor.shutdown(wait=True)`를 호출
- <6> 여러 스레드에 함수를 적용시켜 각 함수가 반환한 값을 가져오는 제너레이터 반환
- <7> 결과 길이 반환 -- 만약 멀티 스레딩 중 에러 발생했으면 암묵적으로 호출된 `next()`에서 에러가 발생

### 17.1.3 Future는 어디에 있나?

`Future`는 `concurrent.futures`와 `asyncio`의 내부에 있는 핵심 컴퍼넌트 입니다. 암묵적으로 `Future`를 사용하지만 위의 예제에서는 드러나있지는 않습니다.

파이썬 3.4 표준 라이브러리에서 `Future`라는 이름을 가진 클래스는 `concurrent.futures.Future`와 `asyncio.Future`가 있습니다. 두 클래스의 객체는 지연된 계산을 표현하기 위해 사용됩니다. 

`Future`는 대기 중인 작업을 큐에 넣고, 완료 상태를 조사하고, 결과를 가져올 수 있도록 캡슐화합니다. 

#### 일반적으로 `Future` 객체를 직접 생성하면 안됩니다. 
`Future` 객체는 `concurrent.futures`나 `asyncio` 같은 동시성 프레임워크에서만 배타적으로 생성해야 합니다. 

따라서 `Future` 객체는 `Executor`의 서브클래스로 실행을 스케줄링한 후에만 생성됩니다.
> `Executor.submit()` 메서드는 콜러블을 받아서, 이 콜러블의 실행을 스케줄링하고, `Future` 객체를 반환합니다. 

#### `Future` 클래스는 논블로킹 방식입니다.  ( `done()` 메서드 )

- 논블로킹(Non-blocking)

논블러킹은 말 그대로 중단되지 않는다는 말. 통신이 완료 될 때까지 중단되는 블로킹의 반대 개념이다.

논블로킹 방식은 아무래도 통신이 완료될 때까지 기다리지 않고 다른 작업을 수행할 수 있으므로 경우에 따라 효율이나 반응속도가 더 뛰어나다.

이 객체에 연결된 콜러블의 실행이 완료되었는지 여부를 불리언형으로 반환합니다. (`done()` 메서드)

#### `result()` 메서드

수행이 완료된 경우 콜러블의 결과를 반환하거나, 콜러블이 실행될 때 발생한 예외를 발생시킵니다. 

- `concurrency.futures.Future` : `f.result()` 는 결과가 나올 때까지 호출자의 스레드를 블로킹합니다. 선택적으로 `timeout` 인수를 전달할 수 있으며, 지정한 시간이 초과하면 `TimeoutError`를 발생시킵니다.
- `asyncio.Future` : 시간초과를 지원하지 않으며 `yield from`을 사용해서 `Future` 객체의 상태를 가져오는 방법을 선호합니다.


#### `Future` 객체를 반환하는 함수

`Fututre` 객체를 사용자에게 보이지 않고 내부에서 사용합니다. 

내부에서 `Future`객체를 사용하는 `Excutor.map()` 메서드는 반복형 객체를 반환하고 `__next__()` 메서드가 호출될 때마다 `Future` 객체의 `result()` 메서드를 호출하므로 `Future`객체의 결과를 가져올 수 있게 해줍니다.

실제로 `Future` 객체를 보기 위해 `concurrent.futures.as_completed()` 함수를 사용하면 다음과 같습니다.

In [7]:
! cat codes/flags_threadpool_ac.py

"""Download flags of top 20 countries by population

ThreadPoolExecutor version 2, with ``as_completed``.

Sample run::

    $ python3 flags_threadpool.py
    BD retrieved.
    EG retrieved.
    CN retrieved.
    ...
    PH retrieved.
    US retrieved.
    IR retrieved.
    20 flags downloaded in 0.93s

"""
from concurrent import futures

from flags import save_flag, get_flag, show, main

MAX_WORKERS = 20


def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


# BEGIN FLAGS_THREADPOOL_AS_COMPLETED
def download_many(cc_list):
    cc_list = cc_list[:5]  # <1>
    with futures.ThreadPoolExecutor(max_workers=3) as executor:  # <2>
        to_do = []
        for cc in sorted(cc_list):  # <3>
            future = executor.submit(download_one, cc)  # <4>
            to_do.append(future)  # <5>
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))  # <6>

        results = []
        for future in futu

- <3> 결과의 순서가 바뀌는 것을 확인해보기 위해 알파벳 순으로 정렬
- <4> `executor.submit()`은 콜러블이 실행되도록 스케줄링하고 이 작업을 나타내는 `Future` 객체를 반환합니다.
- <7> `as_completed()`는 `Future`가 완료될 때 해당 `Future`객체를 생성합니다. 
- <8> `Future` 객체의 결과를 가져와서  <9> `Future` 객체와 결과를 출력합니다.

#### 결과

```
$ python3 flags_threadpool_ac.py
Scheduled for BR: <Future at 0x100791518 state=running> <1>
Scheduled for CN: <Future at 0x100791710 state=running>
Scheduled for ID: <Future at 0x100791a90 state=running>
Scheduled for IN: <Future at 0x101807080 state=pending> <2>
Scheduled for US: <Future at 0x101807128 state=pending>
CN <Future at 0x100791710 state=finished returned str> result: 'CN' <3>
BR ID <Future at 0x100791518 state=finished returned str> result: 'BR' <4>
<Future at 0x100791a90 state=finished returned str> result: 'ID'
IN <Future at 0x101807080 state=finished returned str> result: 'IN' 
US <Future at 0x101807128 state=finished returned str> result: 'US'

5 flags downloaded in 0.70s
```

- <1>  `FUTURE` 객체가 알파벳순으로 스케줄링, `max_workers=3`으로 지정했기 때문에 처음 세개만 실행 중
- <2> 마지막 두 개의 `Future` 객체는 작업 스레드를 기다립니다.
- <3> 멀티 스레딩을 했기 때문에 결과는 순서가 일정하지 않은 것을 볼 수 있습니다. 앞글자 CN은 작업자 스레드에 있는 `download_one()`이 출력한 메시지이고 그 뒤부터 `download_many()`가 출력한 것
- <4> 주 스레드의 `download_many()` 에서 첫 스레드 BR의 결과를 출력하기 전에 BR과 ID 스레드가 국가 코드를 먼저 출력

하지만 엄격히 말하면, 지금까지 테스트한 동시성 스크립트는 병렬로 수행하는 것이 아닙니다. 왜냐하면 `concurrent.futures`는 GIL(전역 인터프리터 락)에 의해 제한되며, `flags_asyncio.py`는 단일 스레드로 실행된다. ( ..? )

- 파이썬 스레드가 한 번에 한 스레드만 실행할 수 있게 해주는 GIL에 의해 제한된다면, 어떻게 `flags_threadpool.py`가 `flags.py`보다 5배나 빠를까?
- 둘 다 단일 스레드인데 어떻게 `asyncio`가 더 빠를까?

두 번째 질문은 18장에서 답변해줍니다.

## 17.2 블로킹 I/O와 GIL

#### GIL
CPython 인터프리터는 내부적으로 스레드가 안전하지 않아서, 전역 인터프리터 락(GIL)을 가지고 있습니다. 

__GIL은 한 번에 한 스레드만 파이썬 바이트코드를 실행하도록 제한합니다.__ 따라서 단일 파이썬 프로세스가 동시에 다중 CPU코어를 사용할 수 없습니다. 

파이썬 코드를 작성할 때 우리는 GIL을 제어할 수 없지만, 내장 함수나 C로 작성된 확장은 시간이 오래 걸리는 작업을 실행할 때 GIL을 해제할 수 있습니다. 

사실 C로 작성된 파이썬 라이브러리는 GIL을 관리하고, 자신의 OS 스레드를 생성해서 가용한 CPU 코드를 모두 사용할 수 있습니다. 하지만 이렇게 하면 라이브러리 코드가 상당히 복잡해지기 때문에, 이런 방식으로 구현하지 않습니다. 

참고 : https://dgkim5360.tistory.com/entry/understanding-the-global-interpreter-lock-of-cpython

#### 블로킹 I/O

블로킹 입출력을 실행하는 모든 표준 라이브러리 함수는 OS에서 결과를 기다리는 동안 GIL을 해제합니다. __즉 입출력 위주의 작업을 실행하는 파이썬 프로그램은 파이썬으로 구현하더라도 스레드를 이용함으로써 이득을 볼 수 있습니다.__

파이썬 스레드가 네트워크로부터의 응답을 기다리는 동안, 블로킹된 입출력 함수가 GIL을 해제함으로써 다른 스레드가 실행될 수 있습니다. 

_파이썬 스레드는 아주 능숙하게 게으름을 피운다. - 데이비드 비즐리_ 

## 17.3 `concurrent.futures`로 프로세스 실행하기

이 `concurrent.futures`는 `ProcessPoolExecutor` 클래스를 사용해서 작업을 여러 파이썬 프로세스에 분산시켜 병렬 컴퓨팅을 가능하게 합니다.

`ProcessPoolExecutor`는 _GIL을 우회하므로 계산 위주의 작업을 수행해야 하는 경우 가용한 CPU를 모두 사용합니다._

> `ProcessPoolExecutor`와 `ThreadPoolExecutor`는 모두 범용 Executor 인터페이스를 구현해서 `concurrent.futures`를 사용하는 경우, 스레드 기반의 프로그램을 프로세스 기반의 프로그램으로 쉽게 변환할 수 있습니다.

국기를 내려받는 예제처럼 입출력 위주의 작업은 멀티 프로세싱을 이용해도 도움이 안되는데, 멀티 프로세싱을 이용하는 경우 사용하는 프로세스 수에 한계가 있기 때문이다.(쿼드 코어 CPU -> 4개의 프로세스밖에 이용못함. 하지만 멀티 스레딩의 경우 더 많은 스레드를 이용 가능)

이러한 멀티프로세싱 연산은 __계산 위주의 작업에서 진가를 발휘합니다.__

이를 테스트하기 위해 다음 두가지 프로그램을 통해 실험해봅니다.

- `arcfour_futures.py` : RC4 알고리즘을 이용해서 배열 10개를 암호화 복호화
- `sha_future.py` : `hashlib`을 이용해서 배열 10개에 대한 SHA-256 해시를 계산

![결과](../images/17_1.png)

4개의 코어를 사용해서 멀티프로세싱을 하면 2배 정도의 성능 향상을 기대할 수 있습니다. 

순수 파이썬으로 구현된 RC4 예제의 경우 PyPy 인터프리터로 4개의 작업자 프로세스를 사용하면 CPython으로 4개의 작업자 프로세스를 사용하는 경우보다 3.8배 빨라집니다. __따라서 계산 위주의 작업을 수행한다면 PyPy를 사용하는 것이 좋습니다.__